In [1]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn as nn
from torchvision import datasets,transforms
import torch.nn.functional as F
import sys
sys.path.append("../code/")
from dataloader import APPLIANCE_ORDER, get_train_test
from sklearn.metrics import mean_absolute_error
import os
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('../code/')
%matplotlib inline
import itertools
from pathlib import Path

In [5]:
with open('../code/script-cnn.sh', 'a') as the_file:
    the_file.write("#!bin/bash \n")
    for dataset in [3]:
        for fold_num in range(5):
            for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
                for lr in [0.001, 0.01, 0.1, 1]:
                    for iterations in [2000, 4000, 6000, 8000]:
                        line = "python cnn-individual.py {} {} {} {} {} \n".format(dataset, fold_num, appliance, lr, iterations)
                        the_file.write(line)

In [5]:
orders = np.load("../code/baseline/rnn-greedy2-orders.npy").item()

In [10]:
best = np.load("../code/baseline/best_param_nested_rnn.npy").item()

In [2]:
k = list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw']))[0]

In [21]:
" ".join(str(x) for x in k)

'hvac fridge dr dw mw'

In [7]:
gid = 0
srv = 1
c = 0
dataset = 5
for fold_num in range(5):
    for lr in [0.01]:
        for iters in [20000]:
            for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw', 'residual'])):
                for p in [0.2, 0.4, 0.8]:
                    o = "\', \'".join(str(x) for x in order)
                    
                    directory = "../code/baseline/cnn-tree/{}/{}/{}/20000/{}/".format(dataset, fold_num, lr, p)
                    filename = "valid-pred-[\'{}\'].npy".format(o)
                    
                    full_path = directory + filename
                    my_file = Path(full_path)
#                     print(full_path)
                    if not my_file.exists():
                        o = " ".join(str(x) for x in order)
                        line = ("CUDA_VISIBLE_DEVICES={} python cnn-tree.py 5 {} {} {} {} {} &".format(gid, lr, iters, p, fold_num, o))
                        print(line)
                        c+=1

#                         with open('../code/0{}{}.sh'.format(srv, gid), 'a') as the_file:
#                             the_file.write("#!bin/bash \ls -l *.outn")
#                             the_file.write(line)

                        if c%1 == 0:
                            gid += 1
                        if gid == 4:
                            gid = 0
                            srv += 1
                    else:
                        continue
print(c, srv, gid)

CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.8 0 hvac mw fridge residual dr dw &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.2 0 hvac mw fridge residual dw dr &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.8 0 hvac mw dr dw fridge residual &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 5 0.01 20000 0.2 0 hvac mw dr dw residual fridge &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.8 0 hvac mw dr dw residual fridge &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.2 0 hvac mw dr residual fridge dw &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.4 0 hvac mw dr residual fridge dw &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 5 0.01 20000 0.2 0 hvac mw dr residual dw fridge &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.4 0 hvac mw dr residual dw fridge &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.8 0 hvac mw dr residual dw fridge &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.2 0 hvac mw d

CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.8 0 fridge mw dw hvac residual dr &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 5 0.01 20000 0.2 0 fridge mw dw dr hvac residual &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.4 0 fridge mw dw dr hvac residual &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.8 0 fridge mw dw dr hvac residual &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.2 0 fridge mw dw dr residual hvac &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 5 0.01 20000 0.4 0 fridge mw dw dr residual hvac &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.8 0 fridge mw dw dr residual hvac &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.2 0 fridge mw dw residual hvac dr &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.4 0 fridge mw dw residual hvac dr &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 5 0.01 20000 0.8 0 fridge mw dw residual hvac dr &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.2 0 fridge mw

CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.4 0 dr residual dw fridge hvac mw &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.8 0 dr residual dw fridge hvac mw &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.8 0 dr residual dw fridge mw hvac &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 5 0.01 20000 0.2 0 dr residual dw mw hvac fridge &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.4 0 dr residual dw mw hvac fridge &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.8 0 dr residual dw mw hvac fridge &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.2 0 dr residual dw mw fridge hvac &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 5 0.01 20000 0.4 0 dr residual dw mw fridge hvac &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.8 0 dr residual dw mw fridge hvac &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.2 0 dr residual mw hvac fridge dw &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.4 0 dr residu

CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.2 0 dw residual fridge dr hvac mw &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.8 0 dw residual fridge mw hvac dr &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.2 0 dw residual mw hvac dr fridge &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 5 0.01 20000 0.4 0 dw residual mw hvac dr fridge &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.8 0 dw residual mw hvac dr fridge &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.2 0 dw residual mw fridge hvac dr &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.4 0 dw residual mw fridge hvac dr &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 5 0.01 20000 0.8 0 dw residual mw fridge hvac dr &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.2 0 dw residual mw fridge dr hvac &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.4 0 dw residual mw fridge dr hvac &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.8 0 dw residu

CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.2 0 residual hvac fridge dr dw mw &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.4 0 residual hvac fridge dr dw mw &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.8 0 residual hvac fridge dr dw mw &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 5 0.01 20000 0.2 0 residual hvac fridge dr mw dw &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.4 0 residual hvac fridge dr mw dw &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.8 0 residual hvac fridge dr mw dw &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.2 0 residual hvac fridge dw dr mw &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 5 0.01 20000 0.2 0 residual hvac dr mw fridge dw &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.4 0 residual hvac dr mw fridge dw &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.8 0 residual hvac dr mw fridge dw &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.2 0 residual 

...

CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.8 3 dr hvac dw residual mw fridge &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.2 3 dr hvac mw fridge dw residual &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 5 0.01 20000 0.4 3 dr hvac mw fridge dw residual &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.8 3 dr hvac mw fridge dw residual &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.2 3 dr hvac mw fridge residual dw &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.4 3 dr hvac mw fridge residual dw &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 5 0.01 20000 0.8 3 dr hvac mw fridge residual dw &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.2 3 dr hvac mw dw fridge residual &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.4 3 dr hvac mw dw fridge residual &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.8 3 dr hvac mw dw fridge residual &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 5 0.01 20000 0.2 3 dr hvac m

CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 5 0.01 20000 0.2 3 dw fridge hvac mw residual dr &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.4 3 dw fridge hvac mw residual dr &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.8 3 dw fridge hvac mw residual dr &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.2 3 dw fridge hvac residual dr mw &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 5 0.01 20000 0.4 3 dw fridge hvac residual dr mw &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.8 3 dw fridge hvac residual dr mw &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.2 3 dw fridge hvac residual mw dr &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.4 3 dw fridge hvac residual mw dr &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 5 0.01 20000 0.8 3 dw fridge hvac residual mw dr &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.2 3 dw fridge dr hvac mw residual &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.4 3 dw fridge

CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 5 0.01 20000 0.2 3 mw hvac dr fridge dw residual &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.4 3 mw hvac dr fridge dw residual &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.8 3 mw hvac dr fridge dw residual &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.2 3 mw hvac dr fridge residual dw &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 5 0.01 20000 0.4 3 mw hvac dr fridge residual dw &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.8 3 mw hvac dr fridge residual dw &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.2 3 mw hvac dr dw fridge residual &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.4 3 mw hvac dr dw fridge residual &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 5 0.01 20000 0.8 3 mw hvac dr dw fridge residual &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.2 3 mw hvac dr dw residual fridge &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.4 3 mw hvac d

CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.4 4 hvac mw residual fridge dw dr &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.8 4 hvac mw residual fridge dw dr &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.8 4 hvac mw residual dw dr fridge &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 5 0.01 20000 0.2 4 hvac residual fridge dr dw mw &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.4 4 hvac residual fridge dr dw mw &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.8 4 hvac residual fridge dr dw mw &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.2 4 hvac residual fridge dr mw dw &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 5 0.01 20000 0.4 4 hvac residual fridge dr mw dw &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.8 4 hvac residual fridge dr mw dw &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.2 4 hvac residual fridge dw dr mw &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.4 4 hvac resi

CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.2 4 residual fridge dw hvac dr mw &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.4 4 residual fridge dw hvac dr mw &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 5 0.01 20000 0.8 4 residual fridge dw hvac dr mw &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.2 4 residual fridge dw hvac mw dr &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.4 4 residual fridge dw hvac mw dr &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.8 4 residual fridge dw hvac mw dr &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 5 0.01 20000 0.4 4 residual fridge mw dr dw hvac &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 5 0.01 20000 0.8 4 residual fridge mw dr dw hvac &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 5 0.01 20000 0.2 4 residual fridge mw dw hvac dr &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 5 0.01 20000 0.4 4 residual fridge mw dw hvac dr &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 5 0.01 20000 0.8 4 residual 

In [17]:
gid = 0
srv = 3
c = 0
cmd = {}
valid_pred = {}
num_iterations = 20000
lr = 0.01
p = 0
dataset = 3
for fold_num in range(5):
    for lr in [0.01]:
        for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw', 'residual'])):


            o = "\', \'".join(str(x) for x in order)
            directory = "../code/baseline/cnn-tree/{}/{}/{}/20000/0.0/".format(dataset, fold_num, lr)
            filename = "valid-pred-[\'{}\'].npy".format(o)

            full_path = directory + filename
            my_file = Path(full_path)
            if not my_file.exists():
                o = " ".join(str(x) for x in order)
                line = ("CUDA_VISIBLE_DEVICES={} python cnn-tree.py 5 {} {} 0 {} {} &".format(gid, lr, num_iterations, fold_num, o))
#                 print(line)

#                 with open('../code/0{}{}.sh'.format(srv, gid), 'a') as the_file:
#                     the_file.write("#!bin/bash \n")
#                     the_file.write(line)
                c += 1
                if c%1 == 0:
                    gid += 1
                if gid == 4:
                    gid = 0
                    srv += 1
            else:
                continue

print(c)                               

3600


In [11]:
print(srv, gid)

12 0


In [13]:
for i in range(1, 12):
    for j in range(4):
        print("nohup sh 0{}{}.sh &> 0{}{}.out &".format(i, j, i, j))

nohup sh 010.sh &> 010.out &
nohup sh 011.sh &> 011.out &
nohup sh 012.sh &> 012.out &
nohup sh 013.sh &> 013.out &
nohup sh 020.sh &> 020.out &
nohup sh 021.sh &> 021.out &
nohup sh 022.sh &> 022.out &
nohup sh 023.sh &> 023.out &
nohup sh 030.sh &> 030.out &
nohup sh 031.sh &> 031.out &
nohup sh 032.sh &> 032.out &
nohup sh 033.sh &> 033.out &
nohup sh 040.sh &> 040.out &
nohup sh 041.sh &> 041.out &
nohup sh 042.sh &> 042.out &
nohup sh 043.sh &> 043.out &
nohup sh 050.sh &> 050.out &
nohup sh 051.sh &> 051.out &
nohup sh 052.sh &> 052.out &
nohup sh 053.sh &> 053.out &
nohup sh 060.sh &> 060.out &
nohup sh 061.sh &> 061.out &
nohup sh 062.sh &> 062.out &
nohup sh 063.sh &> 063.out &
nohup sh 070.sh &> 070.out &
nohup sh 071.sh &> 071.out &
nohup sh 072.sh &> 072.out &
nohup sh 073.sh &> 073.out &
nohup sh 080.sh &> 080.out &
nohup sh 081.sh &> 081.out &
nohup sh 082.sh &> 082.out &
nohup sh 083.sh &> 083.out &
nohup sh 090.sh &> 090.out &
nohup sh 091.sh &> 091.out &
nohup sh 092.s

In [6]:
gid = 0
srv = 1
c = 0


for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw'])):
    o = " ".join(str(x) for x in order)
    line = ("CUDA_VISIBLE_DEVICES={} python cnn-rnn-tree.py 1 GRU 20 1 True 0.01 5000 0 0 {}\n".format(gid, o))
    c+=1

    with open('../code/0{}{}-tree.sh'.format(srv, gid), 'a') as the_file:
        the_file.write("#!bin/bash \n")
        the_file.write(line)

    if c%60 == 0:
        gid += 1
    if gid == 4:
        gid = 0
        srv += 1
            
print(c)


120


In [7]:
print(gid, srv)

2 1


In [3]:
gid = 0
srv = 1
c = 0



for fold_num in range(5):
    for cell_type in ['GRU', 'LSTM', 'RNN']:
        for hidden_size in [20, 50, 100]:
            for num_layers in [1, 2, 3, 4]:
                for bidirectional in [True, False]:
                    for lr in [0.001, 0.01, 0.1, 1]:
                        for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw'])):
                            o = " ".join(str(x) for x in order)
                            line = ("CUDA_VISIBLE_DEVICES={} python rnn-nested-cv.py 2 {} {} {} {} {} 3000 0 {} {}\n".format(gid, cell_type, hidden_size, num_layers, bidirectional, lr, fold_num, o))
                            c+=1
 
#                             with open('../code/0{}{}.sh'.format(srv, gid), 'a') as the_file:
#                                 the_file.write("#!bin/bash \n")
#                                 the_file.write(line)

                            if c%7200 == 0:
                                gid += 1
                            if gid == 4:
                                gid = 0
                                srv += 1

In [4]:
c

172800

In [36]:
app_list = ['hvac', 'fridge', 'dr', 'dw', 'mw']

In [41]:
" ".join(order)

'hvac fridge'

In [43]:
order = ['hvac', 'fridge']
fold_num = 0
for app in app_list:
    if app in order:
        continue
    else:
        print (app)
        
        print ("CUDA_VISIBLE_DEVICES={} python rnn-nested-cv.py {} 2 GRU 20 1 True 0.01 3000 0 {} {}".format(gid, fold_num, " ".join(order), app))

dr
CUDA_VISIBLE_DEVICES=0 python rnn-nested-cv.py 0 2 GRU 20 1 True 0.01 3000 0 hvac fridge dr
dw
CUDA_VISIBLE_DEVICES=0 python rnn-nested-cv.py 0 2 GRU 20 1 True 0.01 3000 0 hvac fridge dw
mw
CUDA_VISIBLE_DEVICES=0 python rnn-nested-cv.py 0 2 GRU 20 1 True 0.01 3000 0 hvac fridge mw


In [54]:
import random

gid=0
for i in range(20):
    index = random.sample(list(np.arange(120)), 1)
#     print(index)
    order = list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw']))[index[0]]
    o = " ".join(str(x) for x in order)
    print ("CUDA_VISIBLE_DEVICES={} python rnn-nested-cv.py 0 1 GRU 50 1 True 0.1 2000 0.4 {}".format(gid, o))
    gid+= 1
    if gid == 4:
        gid = 0
    

CUDA_VISIBLE_DEVICES=0 python rnn-nested-cv.py 0 1 GRU 50 1 True 0.1 2000 0.4 fridge hvac dw mw dr
CUDA_VISIBLE_DEVICES=1 python rnn-nested-cv.py 0 1 GRU 50 1 True 0.1 2000 0.4 dr dw mw hvac fridge
CUDA_VISIBLE_DEVICES=2 python rnn-nested-cv.py 0 1 GRU 50 1 True 0.1 2000 0.4 mw hvac dr dw fridge
CUDA_VISIBLE_DEVICES=3 python rnn-nested-cv.py 0 1 GRU 50 1 True 0.1 2000 0.4 dw mw dr fridge hvac
CUDA_VISIBLE_DEVICES=0 python rnn-nested-cv.py 0 1 GRU 50 1 True 0.1 2000 0.4 fridge dw mw dr hvac
CUDA_VISIBLE_DEVICES=1 python rnn-nested-cv.py 0 1 GRU 50 1 True 0.1 2000 0.4 hvac fridge dr dw mw
CUDA_VISIBLE_DEVICES=2 python rnn-nested-cv.py 0 1 GRU 50 1 True 0.1 2000 0.4 hvac mw fridge dr dw
CUDA_VISIBLE_DEVICES=3 python rnn-nested-cv.py 0 1 GRU 50 1 True 0.1 2000 0.4 dr dw hvac fridge mw
CUDA_VISIBLE_DEVICES=0 python rnn-nested-cv.py 0 1 GRU 50 1 True 0.1 2000 0.4 mw dw dr hvac fridge
CUDA_VISIBLE_DEVICES=1 python rnn-nested-cv.py 0 1 GRU 50 1 True 0.1 2000 0.4 mw hvac dr dw fridge
CUDA_VISIB

In [9]:
gid = 0
for fold_num in range(5):
    for cell_type in ['GRU','LSTM', 'RNN']:
        for hidden_size  in [20, 50]:
            for num_layer in [1, 2, 3]:
                for lr in [0.1, 0.2, 0.5]:
                    
                    directory = "../code/baseline/cnn-rnn-tree/5/{}/{}/{}/{}/True/{}/3000/0.0/".format(fold_num, cell_type, hidden_size, num_layer, lr)
                    filename = "test-error-['mw', 'dw', 'dr', 'fridge', 'hvac'].npy"
                    try:
                        k = np.load(directory+filename)
                    except:
                        
                        print("CUDA_VISIBLE_DEVICES={} python cnn-rnn-tree.py 3 {} {} {} True {} 3000 0 {} mw dw dr fridge hvac &".format(gid, cell_type, hidden_size, num_layer, lr, fold_num))
                        gid+= 1
                        if gid == 4:
                            gid = 0

CUDA_VISIBLE_DEVICES=0 python cnn-rnn-tree.py 3 GRU 50 3 True 0.1 3000 0 0 mw dw dr fridge hvac &
CUDA_VISIBLE_DEVICES=1 python cnn-rnn-tree.py 3 GRU 50 3 True 0.2 3000 0 0 mw dw dr fridge hvac &
CUDA_VISIBLE_DEVICES=2 python cnn-rnn-tree.py 3 GRU 50 3 True 0.5 3000 0 0 mw dw dr fridge hvac &
CUDA_VISIBLE_DEVICES=3 python cnn-rnn-tree.py 3 RNN 50 2 True 0.1 3000 0 0 mw dw dr fridge hvac &
CUDA_VISIBLE_DEVICES=0 python cnn-rnn-tree.py 3 RNN 50 2 True 0.2 3000 0 0 mw dw dr fridge hvac &
CUDA_VISIBLE_DEVICES=1 python cnn-rnn-tree.py 3 RNN 50 2 True 0.5 3000 0 0 mw dw dr fridge hvac &
CUDA_VISIBLE_DEVICES=2 python cnn-rnn-tree.py 3 RNN 50 3 True 0.1 3000 0 0 mw dw dr fridge hvac &
CUDA_VISIBLE_DEVICES=3 python cnn-rnn-tree.py 3 RNN 50 3 True 0.2 3000 0 0 mw dw dr fridge hvac &
CUDA_VISIBLE_DEVICES=0 python cnn-rnn-tree.py 3 GRU 20 1 True 0.5 3000 0 1 mw dw dr fridge hvac &
CUDA_VISIBLE_DEVICES=1 python cnn-rnn-tree.py 3 GRU 20 2 True 0.1 3000 0 1 mw dw dr fridge hvac &
CUDA_VISIBLE_DEVICES

In [13]:
for fold_num in range(5):
    for cell_type in ['GRU', 'LSTM', 'RNN']:
        for hidden_size in [20, 50, 100]:
            for num_layers in [1, 2, 3, 4]:
                for bidirectional in [True,False]:
                    for lr in [0.01, 0.1, 1.0]:
                        print("mkdir -p {}/{}/{}/{}/{}/{}/0.0".format(fold_num, cell_type, hidden_size, num_layers, bidirectional, lr))
                        print("mv *{}-1-{}-{}-{}-{}-{}* ./{}/{}/{}/{}/{}/{}/0.0".format(fold_num, cell_type,hidden_size,num_layers, bidirectional, lr, fold_num, cell_type,hidden_size,num_layers, bidirectional, lr))

mkdir -p 0/GRU/20/1/True/0.01/0.0
mv *0-1-GRU-20-1-True-0.01* ./0/GRU/20/1/True/0.01/0.0
mkdir -p 0/GRU/20/1/True/0.1/0.0
mv *0-1-GRU-20-1-True-0.1* ./0/GRU/20/1/True/0.1/0.0
mkdir -p 0/GRU/20/1/True/1.0/0.0
mv *0-1-GRU-20-1-True-1.0* ./0/GRU/20/1/True/1.0/0.0
mkdir -p 0/GRU/20/1/False/0.01/0.0
mv *0-1-GRU-20-1-False-0.01* ./0/GRU/20/1/False/0.01/0.0
mkdir -p 0/GRU/20/1/False/0.1/0.0
mv *0-1-GRU-20-1-False-0.1* ./0/GRU/20/1/False/0.1/0.0
mkdir -p 0/GRU/20/1/False/1.0/0.0
mv *0-1-GRU-20-1-False-1.0* ./0/GRU/20/1/False/1.0/0.0
mkdir -p 0/GRU/20/2/True/0.01/0.0
mv *0-1-GRU-20-2-True-0.01* ./0/GRU/20/2/True/0.01/0.0
mkdir -p 0/GRU/20/2/True/0.1/0.0
mv *0-1-GRU-20-2-True-0.1* ./0/GRU/20/2/True/0.1/0.0
mkdir -p 0/GRU/20/2/True/1.0/0.0
mv *0-1-GRU-20-2-True-1.0* ./0/GRU/20/2/True/1.0/0.0
mkdir -p 0/GRU/20/2/False/0.01/0.0
mv *0-1-GRU-20-2-False-0.01* ./0/GRU/20/2/False/0.01/0.0
mkdir -p 0/GRU/20/2/False/0.1/0.0
mv *0-1-GRU-20-2-False-0.1* ./0/GRU/20/2/False/0.1/0.0
mkdir -p 0/GRU/20/2/False/1